# 📦 Setup & Imports

In [41]:
import pandas as pd
import rdflib
import glob
import os

# Define the final schema columns
FINAL_COLUMNS = ['Date', 'City', 'Route', 'Mode', 'DayType', 'Ridership']

### 🟩 STORY 2.1 — CTA RDF → CSV (Chicago)

This handles the 19 RDF files. The predicates (the XML tags) we are looking for are ds:date, ds:route, ds:daytype, and ds:rides.

In [42]:
# daily RDF to DataFrame
from rdflib import Graph


def rdf_daily_to_df(file_path):
    g = Graph()
    g.parse(file_path, format="xml")

    rows = {}

    for s, p, o in g:
        s = str(s)
        p = str(p)
        o = str(o)

        if s not in rows:
            rows[s] = {
                "subject": s,
                "city": "Chicago"
            }

        col = p.split("/")[-1]

        if col == "route":
            rows[s]["route"] = o
        elif col == "date":
            rows[s]["date"] = o
        elif col == "daytype":
            rows[s]["daytype"] = o
        elif col == "rides":
            rows[s]["ridership"] = o

    df = pd.DataFrame(rows.values())

    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df["ridership"] = pd.to_numeric(df["ridership"], errors="coerce")

    return df

dfs = []
for file in os.listdir("../data/raw/RDF_CTA/"):
    if file.endswith(".rdf"):
        full_path = os.path.join("../data/raw/RDF_CTA/", file)
        print(f"Reading: {file}")
        df_temp = rdf_daily_to_df(full_path)
        dfs.append(df_temp)


df_Daily = pd.concat(dfs, ignore_index=True)

Reading: rdf_CTA__Ridership__Daily_by_Route_routes_1.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_10.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_11.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_12.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_13.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_14.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_15.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_16.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_17.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_18.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_19.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_2.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_3.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_4.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_5.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_6.rdf
Reading: rdf_CTA__Ridership__Daily_by_Route_routes_7.rdf
Reading: rdf_CTA__Rid

In [43]:
df_Daily.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1092474 entries, 0 to 1092473
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype         
---  ------     --------------    -----         
 0   subject    1092474 non-null  object        
 1   city       1092474 non-null  object        
 2   ridership  1092474 non-null  int64         
 3   route      1092474 non-null  object        
 4   date       1092474 non-null  datetime64[ns]
 5   daytype    1092474 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 50.0+ MB


In [44]:
df_Daily.to_csv(R"../data/processed/cta_daily_ridership.csv", index=False)

### 🟩 STORY 2.2 — Chicago Excel (CSV) Cleaning

You uploaded the "Daily Boarding Totals" as a CSV. We need to align it with the RDF schema.

In [45]:
# Task 2.2.1 & 2.2.2: Load and Clean Chicago Boarding Totals
chicago_totals = pd.read_excel(R"../data/raw/cta-ridership-daily-boarding-totals-20260203-69820a3f9df63091665572.xlsx")

# We need to melt this data because 'bus' and 'rail_boardings' are separate columns 
chicago_melted = chicago_totals.melt(
    id_vars=['service_date', 'day_type'], 
    value_vars=['bus', 'rail_boardings'],
    var_name='Mode', value_name='Ridership'
)

# Harmonizing schema
chicago_melted.rename(columns={'service_date': 'Date', 'day_type': 'DayType'}, inplace=True)
chicago_melted['City'] = 'Chicago'
chicago_melted['Route'] = 'Total' # These are system-wide totals, not specific routes
chicago_melted['Mode'] = chicago_melted['Mode'].replace({'rail_boardings': 'Rail', 'bus': 'Bus'})
chicago_melted['Date'] = pd.to_datetime(chicago_melted['Date']).dt.date

cta_boarding_mode_clean = chicago_melted[FINAL_COLUMNS]
cta_boarding_mode_clean.to_csv(R"../data/processed/cta_boarding_mode_clean.csv", index=False)

### 🟩 STORY 2.3 — Philadelphia CSV Cleaning

This processes the SEPTA data from your uploaded files.

In [46]:
# Task 2.3.1: Philadelphia By Mode
septa_mode = pd.read_csv(R"../data/raw/Average_Daily_Ridership_By_Mode - City of Philadelphia.csv")
septa_mode['City'] = 'Philadelphia'
septa_mode['Route'] = 'Total'
septa_mode.rename(columns={'Calendar_Year': 'Year', 'Calendar_Month': 'Month', 
                           'Average_Daily_Ridership': 'Ridership', 'Mode': 'Mode'}, inplace=True)

# Create a proper date (using first of the month) [cite: 1]
septa_mode['Date'] = pd.to_datetime(septa_mode[['Year', 'Month']].assign(Day=1)).dt.date
septa_mode['DayType'] = 'W' # Standardizing as Weekday average

septa_ridership_mode_clean = septa_mode[FINAL_COLUMNS]

# Task 2.3.2: Philadelphia By Route [cite: 2]
septa_route = pd.read_csv(R"../data/raw/Average_Daily_Ridership_By_Route - City of Philadelphia.csv")

### 🟩 STORY 2.4 — Unified Fact Table

Finally, we glue everything together into one file.

In [48]:
# Task 2.4.1 & 2.4.2: Unification
fact_ridership = pd.concat([
    df_Daily,
    cta_boarding_mode_clean,
    septa_ridership_mode_clean
]).drop_duplicates()

# Final export
fact_ridership.to_csv(R"../data/processed/fact_ridership.csv", index=False)
print("ETL Complete: fact_ridership.csv created with", len(fact_ridership), "rows.")

ETL Complete: fact_ridership.csv created with 1111166 rows.
